<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/hema/Run_Heme_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division, print_function
!rm -r abstention
!git clone https://github.com/blindauth/abstention.git
!pip uninstall abstention
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 127 (delta 75), reused 91 (delta 39), pack-reused 0
Receiving objects: 100% (127/127), 65.16 KiB | 8.15 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-qop0a98l/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 1113, done.
remote: Counting objects: 100% (1113/1113), done.
remote: Compressing objects: 100% (1108/1108), done.
remote: Total 4051 (delta 41), reused 1048 (delta 5), pack-reused 2938
Receiving objects: 100% (4051/4051), 289.33 MiB | 44.92 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [0]:
import gzip
import numpy as np

def read_vals(gzip_file, idx_to_load):
    return np.array([[
        float(y) for y in x.decode("utf-8").rstrip().split("\t")][idx_to_load]
                          for x in gzip.open(gzip_file)])
  
def get_calc_tpr_at_fpr_func(fpr_level): 
    def tpr_at_fpr(y_true, y_score):
      #verify probs are sorted, if not, sort them
      if (np.min(y_score[1:] - y_score[:-1]) < 0):
          (sorted_y_score, sorted_y_true) = zip(*sorted(zip(y_score,y_true),
                                                   key=lambda x: x[0]))
          sorted_y_score = np.array(sorted_y_score)
          sorted_y_true = np.array(sorted_y_true)
      else:
          sorted_probs = y_score
          sorted_y_true = y_true
      pos_cumsum = np.cumsum(sorted_y_true)
      tprs = (pos_cumsum[-1]-pos_cumsum)/pos_cumsum[-1]
      neg_cumsum = np.cumsum(1-sorted_y_true)
      fprs = (neg_cumsum[-1]-neg_cumsum)/neg_cumsum[-1]
      for (tpr,fpr) in zip(tprs,fprs):
          if (fpr <= fpr_level):
              return tpr    
    return tpr_at_fpr
  
def fiftyfiftysplit(indices, seed):
    rng = np.random.RandomState(seed)
    indices_1 = rng.choice(a=indices,
                           size=int(0.5*len(indices)),
                           replace=False)
    set_indices_1 = set(indices_1)
    indices_2 = np.array(
        [i for i in indices if i not in set_indices_1])
    return (indices_1, indices_2)

In [4]:
import numpy as np
import random
from abstention.abstention import (
    MonteCarloWindowAbstDeltaTprAtFprThreshold,
    EstWindowAbstDeltaAuroc,
    MonteCarloWindowAbstDeltaAuprc,
    EstWindowAbstDeltaAuprc,
    MonteCarloSubsampleNaiveEval,
    Uncertainty, NegPosteriorDistanceFromThreshold,
    FixedThreshold, OneMinusJSDivFromClassFreq,
    DualThresholdsFromPointFiveOnValidSet,
    assertsorted_average_precision_score, basic_average_precision_score)
from abstention.calibration import (Expit, PlattScaling)
from sklearn.metrics import roc_auc_score
from collections import OrderedDict
import sys
import json
import os


model_dirs = ["record_12_model_bo6Z3_preds",
    "record_30_model_vo6bz_preds",
    "record_15_model_Bscof_preds",
    "record_3_model_e0JNc_preds",
    "record_18_model_xZ1BL_preds",
    "record_6_model_T8zn7_preds",
    "record_21_model_ovIXJ_preds",
    "record_9_model_8pkw7_preds",
    "record_24_model_7n9bs_preds",
    "record_27_model_3QpKH_preds"
             ]

pseudotest_split_size = int(0.5*len(read_vals(
                       "abstention_experiments/hema/test_labels.txt.gz",
                        idx_to_load=0)))
target_fpr_level = 0.2
n_samples=100
smoothing_window_size=11

metricname_to_computefunc = OrderedDict([
    ('auroc', roc_auc_score),
    ('auprc', basic_average_precision_score),
    ('tpratfpr_'+str(target_fpr_level),
     get_calc_tpr_at_fpr_func(target_fpr_level))
])

abstname_to_factory = OrderedDict([
    ('mcwindowdelta_tpratfpr_30pc',
     MonteCarloWindowAbstDeltaTprAtFprThreshold(
         n_samples=n_samples, fpr_threshold=target_fpr_level,
         num_to_abstain_on=int(0.3*pseudotest_split_size),
         smoothing_window_size=smoothing_window_size,
         return_max_across_windows=True)),
    ('minibatch_tpratfpr_30pc', MonteCarloSubsampleNaiveEval(
            metric=get_calc_tpr_at_fpr_func(target_fpr_level),
            num_to_abstain_on=int(0.3*pseudotest_split_size),
            num_to_subsample=int(np.sqrt(pseudotest_split_size)),
            return_max_across_windows=True,
            n_samples=n_samples,
            smoothing_window_size=smoothing_window_size)),
    ('fumera_tpratfpr_30pc', DualThresholdsFromPointFiveOnValidSet(
                          fracs_to_abstain_on=[0.3],
                          metric=get_calc_tpr_at_fpr_func(target_fpr_level))),
    ('mcwindowdelta_tpratfpr_15pc',
     MonteCarloWindowAbstDeltaTprAtFprThreshold(
         n_samples=n_samples, fpr_threshold=target_fpr_level,
         num_to_abstain_on=int(0.15*pseudotest_split_size),
         smoothing_window_size=smoothing_window_size,
         return_max_across_windows=True)),
    ('minibatch_tpratfpr_30pc', MonteCarloSubsampleNaiveEval(
            metric=get_calc_tpr_at_fpr_func(target_fpr_level),
            num_to_abstain_on=int(0.15*pseudotest_split_size),
            num_to_subsample=int(np.sqrt(pseudotest_split_size)),
            return_max_across_windows=True,
            n_samples=n_samples,
            smoothing_window_size=smoothing_window_size)),
    ('fumera_tpratfpr_15pc', DualThresholdsFromPointFiveOnValidSet(
                          fracs_to_abstain_on=[0.15],
                          metric=get_calc_tpr_at_fpr_func(target_fpr_level))),   
    ('js', OneMinusJSDivFromClassFreq()),
    ('estwindowdelta_auroc_30pc',
     EstWindowAbstDeltaAuroc(
         num_to_abstain_on=int(0.3*pseudotest_split_size),
         return_max_across_windows=True)),
    ('estwindowdelta_auroc_15pc',
     EstWindowAbstDeltaAuroc(
         num_to_abstain_on=int(0.15*pseudotest_split_size),
         return_max_across_windows=True)),
    ('fumera_auroc_30pc', DualThresholdsFromPointFiveOnValidSet(
                           fracs_to_abstain_on=[0.3],
                           metric=roc_auc_score)),
    ('fumera_auroc_15pc', DualThresholdsFromPointFiveOnValidSet(
                          fracs_to_abstain_on=[0.15],
                          metric=roc_auc_score)),
    ('estwindowdelta_auprc_30pc',
     EstWindowAbstDeltaAuprc(
         num_to_abstain_on=int(0.3*pseudotest_split_size),
         return_max_across_windows=True)),
    ('estwindowdelta_auprc_15pc',
     EstWindowAbstDeltaAuprc(
         num_to_abstain_on=int(0.15*pseudotest_split_size),
         return_max_across_windows=True)),
    ('mcwindowdelta_auprc_30pc',
     MonteCarloWindowAbstDeltaAuprc(
         num_to_abstain_on=int(0.3*pseudotest_split_size),
         return_max_across_windows=True,
            n_samples=n_samples,
            smoothing_window_size=smoothing_window_size)),
    ('mcwindowdelta_auprc_15pc',
     MonteCarloWindowAbstDeltaAuprc(
         num_to_abstain_on=int(0.15*pseudotest_split_size),
         return_max_across_windows=True,
            n_samples=n_samples,
            smoothing_window_size=smoothing_window_size)),
    ('minibatch_auprc_30pc', MonteCarloSubsampleNaiveEval(
            metric=assertsorted_average_precision_score,
            num_to_abstain_on=int(0.3*pseudotest_split_size),
            num_to_subsample=int(np.sqrt(pseudotest_split_size)),
            return_max_across_windows=True,
            n_samples=n_samples,
            smoothing_window_size=smoothing_window_size)),
    ('minibatch_auprc_15pc', MonteCarloSubsampleNaiveEval(
            metric=assertsorted_average_precision_score,
            num_to_abstain_on=int(0.15*pseudotest_split_size),
            num_to_subsample=int(np.sqrt(pseudotest_split_size)),
            return_max_across_windows=True,
            n_samples=n_samples,
            smoothing_window_size=smoothing_window_size)),
    ('fumera_auprc_30pc', DualThresholdsFromPointFiveOnValidSet(
                           fracs_to_abstain_on=[0.3],
                           metric=assertsorted_average_precision_score)),
    ('fumera_auprc_15pc', DualThresholdsFromPointFiveOnValidSet(
                          fracs_to_abstain_on=[0.15],
                          metric=assertsorted_average_precision_score)),
    ('uncertainty', Uncertainty()),
    ('distfrom0.5', NegPosteriorDistanceFromThreshold(FixedThreshold(0.5))),  
])

calibname_to_factory = OrderedDict([
    ('none', Expit()),
    ('platt', PlattScaling(verbose=False))
])

abstfracs = [0.15, 0.3]

num_trials_per_model = 3
for task_idx in [0, 1]:
  print("\n###On task idx###", task_idx)  
  
  ###
  #initialize everything
  metricname_to_baselineperfs = OrderedDict()
  for metricname in metricname_to_computefunc:
    metricname_to_baselineperfs[metricname] = []
  metricname_to_abstfrac_to_abstncalib_to_perf = OrderedDict()
  for metricname in metricname_to_computefunc:
    abstfrac_to_abstncalib_to_perf = OrderedDict()
    for abstfrac in abstfracs:
      abstncalib_to_perf = OrderedDict()
      for abstname in abstname_to_factory:
        for calibname in calibname_to_factory:
          abstncalib_to_perf[abstname+":"+calibname] = []
      abstfrac_to_abstncalib_to_perf[abstfrac] = abstncalib_to_perf
    metricname_to_abstfrac_to_abstncalib_to_perf[metricname] = (
      abstfrac_to_abstncalib_to_perf)
  ####    
  
  loaded_test_labels = read_vals(
      "abstention_experiments/hema/test_labels.txt.gz",
      idx_to_load=task_idx)
  print("Test set imbalance:",np.mean(loaded_test_labels))
  
  for (model_idx, model_dir) in enumerate(model_dirs):
    print("\n###On model dir###",model_dir)
    loaded_test_det_preds =\
      read_vals(
        "abstention_experiments/hema/"+model_dir+"/test_det_preds.txt.gz",
        idx_to_load=task_idx)
    loaded_test_nondet_preds = np.array([
      read_vals("abstention_experiments/hema/"+model_dir
                +"/test_nondet_preds_"+str(i)+".txt.gz",
                idx_to_load=task_idx)
                for i in range(100)]).transpose((1,0))
    print("Overall metrics:")
    for metricname in metricname_to_computefunc:
      computefunc = metricname_to_computefunc[metricname]
      print(metricname,":",computefunc(y_true=loaded_test_labels,
                                       y_score=loaded_test_det_preds))
       
    for trial_idx in range(num_trials_per_model):
      print("\n###Trial idx###", trial_idx)
      pseudovalid_indices, pseudotest_indices = fiftyfiftysplit(
        indices=np.arange(len(loaded_test_det_preds)),
                          seed=100*trial_idx + 10*model_idx)
      pseudovalid_labels = loaded_test_labels[pseudovalid_indices]
      pseudovalid_det_preds = loaded_test_det_preds[pseudovalid_indices]
      pseudotest_labels = loaded_test_labels[pseudotest_indices]
      pseudotest_det_preds = loaded_test_det_preds[pseudotest_indices]
      pseudotest_nondet_preds = loaded_test_nondet_preds[pseudotest_indices]    
      
      #compute the baseline test perfs
      for metricname in metricname_to_computefunc:
        computefunc = metricname_to_computefunc[metricname]
        perf = computefunc(y_true=pseudotest_labels,
                           y_score=pseudotest_det_preds)
        metricname_to_baselineperfs[metricname].append(perf)
      
      #perform the calibration
      calibname_to_pseudovalid_detpreds = OrderedDict()
      calibname_to_pseudotest_detpreds = OrderedDict()
      calibname_to_pseudotest_nondetpreds = OrderedDict()
      for calibname in calibname_to_factory:
        calib_factory = calibname_to_factory[calibname]
        calib_func = calib_factory(
            valid_preacts=pseudovalid_det_preds,
            valid_labels=pseudovalid_labels)
        calibname_to_pseudovalid_detpreds[calibname] = calib_func(
            preact=pseudovalid_det_preds)
        calibname_to_pseudotest_detpreds[calibname] = calib_func(
            preact=pseudotest_det_preds)
        calibname_to_pseudotest_nondetpreds[calibname] = (
          np.array([calib_func(x) for x in
                    pseudotest_nondet_preds.transpose((1,0))
                   ]).transpose((1,0)))
      
      for abstname in abstname_to_factory:
        sys.stdout.flush()
        abst_factory = abstname_to_factory[abstname]
        for calibname in calibname_to_factory:
          print("\nOn method", abstname+":"+calibname)
          calib_pseudovalid_detpreds =\
            calibname_to_pseudovalid_detpreds[calibname]
          abstention_func = abst_factory(
              valid_labels=pseudovalid_labels,
              valid_posterior=calib_pseudovalid_detpreds)
          
          #get the abstention priorities, and sort the indices accordingly
          calib_pseudotest_detpreds =\
            calibname_to_pseudotest_detpreds[calibname]
          var_calib_pseudotest_nondetpreds = np.var(
              calibname_to_pseudotest_nondetpreds[calibname], axis=1)
          pseudotest_abstention_priorities =\
            abstention_func(
              posterior_probs=calib_pseudotest_detpreds,
              uncertainties=var_calib_pseudotest_nondetpreds)
          sorted_indices = [
                x[0] for x in
                sorted(enumerate(pseudotest_abstention_priorities),
                key=lambda x: x[1])]          
          
          for abstfrac in abstfracs:
            #get the post-abstention examples            
            num_to_retain = int(len(sorted_indices)*(1-abstfrac))
            retained_test_detpreds = calib_pseudotest_detpreds[
                sorted_indices[:num_to_retain]]
            retained_test_labels =\
              pseudotest_labels[sorted_indices[:num_to_retain]]
            
            for metricname in metricname_to_computefunc:
              metric_compute_func = metricname_to_computefunc[metricname]
              perf = metric_compute_func(y_true=retained_test_labels,
                                         y_score=retained_test_detpreds)
              metricname_to_abstfrac_to_abstncalib_to_perf[metricname]\
                [abstfrac][abstname+":"+calibname].append(perf)
              print("Abstfrac",abstfrac,metricname,perf)

  file_out = "heme_task"+str(task_idx)+"_abstention_results.json"
  dict_to_write = {
      "metricname_to_baselineperfs": metricname_to_baselineperfs,
      "metricname_to_abstfrac_to_abstncalib_to_perf":
       metricname_to_abstfrac_to_abstncalib_to_perf}
  open(file_out, 'w').write(
      json.dumps(dict_to_write,
                 sort_keys=True, indent=4, separators=(',', ': ')))
  os.system("gzip -f "+file_out)


###On task idx### 0
Test set imbalance: 0.3387

###On model dir### record_12_model_bo6Z3_preds
Overall metrics:
auroc : 0.8330560569716421
auprc : 0.7364778896384668
tpratfpr_0.2 : 0.6947150870977266

###Trial idx### 0

On method mcwindowdelta_tpratfpr_30pc:none


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Abstfrac 0.15 auroc 0.8573720527882835
Abstfrac 0.15 auprc 0.7719971192045407
Abstfrac 0.15 tpratfpr_0.2 0.7510259917920656
Abstfrac 0.3 auroc 0.8834194670083674
Abstfrac 0.3 auprc 0.8087431447032575
Abstfrac 0.3 tpratfpr_0.2 0.8197251414713015

On method mcwindowdelta_tpratfpr_30pc:platt
Abstfrac 0.15 auroc 0.8576619753128286
Abstfrac 0.15 auprc 0.772087338217454
Abstfrac 0.15 tpratfpr_0.2 0.7508579272477693
Abstfrac 0.3 auroc 0.8846744288660165
Abstfrac 0.3 auprc 0.8092401374037289
Abstfrac 0.3 tpratfpr_0.2 0.8386831275720165

On method minibatch_tpratfpr_30pc:none
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.8578467402707227
Abstfrac 0.15 auprc 0.7713827474035199
Abstfrac 0.15 tpratfpr_0.2 0.7578726382085375
Abstfrac 0.3 auroc 0.8858054160115275
Abstfrac 0.3 auprc 0.8093855877944176
Abstfrac 0.3 tpratfpr_0.2 0.8299492385786802

On method minibatch_tpratfpr_30pc:platt
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.8579533251231528
Abstfrac 0.15 auprc 0.7721779593700335
Abstfrac 0.15 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Abstfrac 0.15 auroc 0.8585457748692547
Abstfrac 0.15 auprc 0.7693258072646687
Abstfrac 0.15 tpratfpr_0.2 0.7539089055064582
Abstfrac 0.3 auroc 0.8841771318954998
Abstfrac 0.3 auprc 0.8048406047480123
Abstfrac 0.3 tpratfpr_0.2 0.8500810372771475

On method mcwindowdelta_tpratfpr_30pc:platt
Abstfrac 0.15 auroc 0.8594611589198203
Abstfrac 0.15 auprc 0.769548324838934
Abstfrac 0.15 tpratfpr_0.2 0.7623626373626373
Abstfrac 0.3 auroc 0.8852965825428696
Abstfrac 0.3 auprc 0.8051079611839252
Abstfrac 0.3 tpratfpr_0.2 0.8506600660066007

On method minibatch_tpratfpr_30pc:none
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.8594993933695327
Abstfrac 0.15 auprc 0.7691277957471009
Abstfrac 0.15 tpratfpr_0.2 0.7647467036780013
Abstfrac 0.3 auroc 0.8858360465387667
Abstfrac 0.3 auprc 0.8045952105203816
Abstfrac 0.3 tpratfpr_0.2 0.8454391891891891

On method minibatch_tpratfpr_30pc:platt
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.859438383094476
Abstfrac 0.15 auprc 0.7688786777871228
Abstfrac 0.15 t

...

Abstfrac 0.15 auroc 0.8301541966304551
Abstfrac 0.15 auprc 0.7175212816842177
Abstfrac 0.15 tpratfpr_0.2 0.7066942719116632
Abstfrac 0.3 auroc 0.8553752946396231
Abstfrac 0.3 auprc 0.7482984072510052
Abstfrac 0.3 tpratfpr_0.2 0.7921300256629598

On method mcwindowdelta_tpratfpr_30pc:platt
Abstfrac 0.15 auroc 0.830191605299419
Abstfrac 0.15 auprc 0.7181943687110407
Abstfrac 0.15 tpratfpr_0.2 0.7073337902673064
Abstfrac 0.3 auroc 0.8551946069164744
Abstfrac 0.3 auprc 0.7490861898347236
Abstfrac 0.3 tpratfpr_0.2 0.7984822934232715

On method minibatch_tpratfpr_30pc:none
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.8315187289133961
Abstfrac 0.15 auprc 0.7187731083328778
Abstfrac 0.15 tpratfpr_0.2 0.7224913494809688
Abstfrac 0.3 auroc 0.8550009934447473
Abstfrac 0.3 auprc 0.7465931529465663
Abstfrac 0.3 tpratfpr_0.2 0.7809773123909249

On method minibatch_tpratfpr_30pc:platt
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.8314169157654001
Abstfrac 0.15 auprc 0.7197237401231779
Abstfrac 0.15 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Abstfrac 0.15 auroc 0.831622186726899
Abstfrac 0.15 auprc 0.7262181330990213
Abstfrac 0.15 tpratfpr_0.2 0.7132963988919667
Abstfrac 0.3 auroc 0.8565496637317223
Abstfrac 0.3 auprc 0.7586099674342082
Abstfrac 0.3 tpratfpr_0.2 0.7937181663837012

On method mcwindowdelta_tpratfpr_30pc:platt
Abstfrac 0.15 auroc 0.8318732512787149
Abstfrac 0.15 auprc 0.7269724452250264
Abstfrac 0.15 tpratfpr_0.2 0.7143840330350998
Abstfrac 0.3 auroc 0.8563111589079557
Abstfrac 0.3 auprc 0.7594459944751975
Abstfrac 0.3 tpratfpr_0.2 0.798999165971643

On method minibatch_tpratfpr_30pc:none
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.8324159045438573
Abstfrac 0.15 auprc 0.7278885914355393
Abstfrac 0.15 tpratfpr_0.2 0.7141877998629198
Abstfrac 0.3 auroc 0.8555997945841464
Abstfrac 0.3 auprc 0.7556376936663786
Abstfrac 0.3 tpratfpr_0.2 0.7769973661106233

On method minibatch_tpratfpr_30pc:platt
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.8324507496513249
Abstfrac 0.15 auprc 0.7267714661918768
Abstfrac 0.15 t

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Abstfrac 0.15 auroc 0.8179920341733027
Abstfrac 0.15 auprc 0.693093151753825
Abstfrac 0.15 tpratfpr_0.2 0.6754690757470465
Abstfrac 0.3 auroc 0.84246836319331
Abstfrac 0.3 auprc 0.7214549224356199
Abstfrac 0.3 tpratfpr_0.2 0.7757575757575758

On method mcwindowdelta_tpratfpr_30pc:platt
Abstfrac 0.15 auroc 0.8178276606490817
Abstfrac 0.15 auprc 0.694308278621557
Abstfrac 0.15 tpratfpr_0.2 0.6673525377229081
Abstfrac 0.3 auroc 0.8419695497883651
Abstfrac 0.3 auprc 0.7231307836754962
Abstfrac 0.3 tpratfpr_0.2 0.7598989048020219

On method minibatch_tpratfpr_30pc:none
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.8194249270615134
Abstfrac 0.15 auprc 0.6955159458458369
Abstfrac 0.15 tpratfpr_0.2 0.6799724707501721
Abstfrac 0.3 auroc 0.8417333147022276
Abstfrac 0.3 auprc 0.7185714065618
Abstfrac 0.3 tpratfpr_0.2 0.7640249332146037

On method minibatch_tpratfpr_30pc:platt
Subsampled 71 intervals
Abstfrac 0.15 auroc 0.8178780751622479
Abstfrac 0.15 auprc 0.6898902945118391
Abstfrac 0.15 tpratf

In [5]:
!ls

abstention		heme_task0_abstention_results.json.gz  sample_data
abstention_experiments	heme_task1_abstention_results.json.gz


In [0]:
from google.colab import files
files.download("heme_task0_abstention_results.json.gz")
files.download("heme_task1_abstention_results.json.gz")

In [0]:
!ls